In [3]:
print ("Hello Dengu u")

Hello Dengu u


In [4]:
"""Naa istam"""

'Naa istam'

In [5]:
5+5

10

In [6]:
from pyspark.sql import *


In [7]:
import pyspark.mllib

In [8]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [14]:
raw_data = sc.textFile("hour_noheader.csv")

In [15]:
raw_data

hour_noheader.csv MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0

In [16]:
records = raw_data.map(lambda x: x.split(","))

In [18]:
first = records.first()

In [19]:
print (first)

['1', '2011-01-01', '1', '0', '1', '0', '0', '6', '0', '1', '0.24', '0.2879', '0.81', '0', '3', '13', '16']


In [20]:
records.cache()

PythonRDD[5] at RDD at PythonRDD.scala:52

In [21]:
def get_mapping(rdd, idx):
    return rdd.map(lambda fields: fields[idx]).distinct().zipWithIndex().collectAsMap()

In [22]:
print ("Mapping of first categorical feasture column: %s" % (get_mapping(records, 2)))

Mapping of first categorical feasture column: {'1': 0, '4': 1, '2': 2, '3': 3}


In [24]:
mappings = [get_mapping(records, i) for i in range(2,10)]
cat_len = sum(map(len, mappings))
num_len = len(records.first()[11:15])
total_len = num_len + cat_len

In [26]:
print ("Feature vector length for categorical features: %d " % cat_len)
print ("Feature vector length for numerical features: %d " % num_len)
print ("Total feature vector length: %d " % total_len)

Feature vector length for categorical features: 57 
Feature vector length for numerical features: 4 
Total feature vector length: 61 
